# HSBXL Membership Tracker

The goal of this notebook is to establish the financial status of **HSBXL** 
(aka Hackerspace Brussels, http://www.hackerspace.be)

## 1. Import required libraries

In [ ]:
import csv
import glob
from pprint import pprint

import vincent
vincent.core.initialize_notebook()

from datetime import datetime, date
from IPython.core.display import HTML

In [ ]:
def str_to_date(string):
    return datetime.strptime(string, "%d-%m-%Y").date()

## 2. Define parsing functions

In [ ]:
def read_bank_csv(filename):
    with open(filename, encoding='latin1') as csvfile:
        csvfile.readline()
        csvfile.readline()
        reader = csv.reader(csvfile, delimiter=';')
        for row in reader:
            yield row

In [ ]:
# CSV fields in order:
args = ['value_date', 'operation_id', 'description', 'amount', 'currency', 'operation_date', 
        'from_account', 'from_name', 'communication1', 'communication2']

def parse_row(row):
    "Parse a row from the CSV, and convert fields that can be converted (amount, dates)."
    result = {args[i]: row[i] for i in range(len(args))}
    result['amount'] = float(result['amount'].replace('.', '').replace(',', '.'))
    result['value_date'] = str_to_date(result['value_date'])
    result['operation_date'] = str_to_date(result['operation_date'])
    return result

In [ ]:
def list_bank_csvs():
    return glob.glob('bank/*/*.csv')

## 3. Extract transfer info per member

In [ ]:
memberships = {}

for csvfile in list_bank_csvs():
    for row in read_bank_csv(csvfile):
        parsed = parse_row(row)
        
        if parsed['description'] != "Virement en votre faveur":
            # Probably expense, skip it
            continue
        if not parsed['communication1'].startswith('+++'):
            # Probably not a membership, skip it
            continue
        
        # Membership id is +++XXX/XXXX/XXXXX+++ in bank tansferts
        membership_id = parsed['communication1']
        
        if membership_id not in memberships:
            memberships[membership_id] = []
        
        memberships[membership_id].append(parsed)

## 4. Total membership paid per person

In [ ]:
paid_total = {membership[0]['from_name']: sum(transfer['amount'] for transfer in membership)
              for membership in memberships.values()}

paid_total

## 4.b. How much membership did Bob donate so far ?

In [ ]:
for membership in memberships.values():
    if 'BOB' in membership[0]['from_name']:
        pprint(membership)

## 5.a. Money in & out in 2015

In [ ]:
total_in = 0
total_out = 0
max_date = date(1, 1, 1)

for csvfile in list_bank_csvs():
    for row in read_bank_csv(csvfile):
        parsed = parse_row(row)
        if parsed['value_date'] < date(2015, 1, 1):
            continue
        if parsed['value_date'] > max_date:
            max_date = parsed['value_date']
            
        if parsed['amount'] > 0:
            total_in += parsed['amount']
        else:
            total_out += parsed['amount']
            
print("Total in:  ", total_in)
print("     out:", total_out)
print('From january untill', max_date)

## 5.b. Money in from 'clean' membership transfer

Where 'clean' membership only counts members that paid by bank transfert with their +++ code as communication.

Note that some people may have paid at the beginning of the year, and therefore.

In [ ]:
total_membership_in = 0
duration = max_date - date(2015, 1, 1)

for membership in memberships.values():
    for transfer in membership:
        if transfer['value_date'] < date(2015, 1, 1):
            continue
        total_membership_in += transfer['amount']
        
print("Total in from membership:", total_membership_in)
print("in", duration.days, "days")
print("This is", total_membership_in / duration.days, "per day")
print("     or", 30 * total_membership_in / duration.days, "per month" )
print("This represents", 100 * total_membership_in / total_in, "% of all income")